In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
import threading

In [6]:
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [65]:
def get_data(year, player_type, data_type):
  """
  Scrapes data from baseball-reference.com
  player_type is either batting or pitching
  data_type indicates whether it is individual or team data
  """
  driver = webdriver.Chrome(chromedriver)
  url = f"https://www.baseball-reference.com/leagues/majors/{year}-standard-{player_type}.shtml"
  driver.get(url)
  if data_type == "player":
    driver.execute_script("window.scrollTo(0, 1500);")
  driver.find_element_by_xpath(f"//*[@id=\"{data_type}_standard_{player_type}_sh\"]/div/ul/li[1]/span").click()
  # time.sleep(1)
  driver.find_element_by_xpath(f"//*[@id=\"{data_type}_standard_{player_type}_sh\"]/div/ul/li[1]/div/ul/li[3]/button").click()
  soup = BeautifulSoup(driver.page_source, "html.parser")
  text_data = "\n".join(soup.find("pre", id=f"csv_{data_type}_standard_{player_type}").text.split("\n")[4:-1])
  with open(f"data/{data_type}_data/{player_type}{year}.csv", "w") as f:
    f.write(text_data)
  driver.quit()


In [42]:
threads = []
year_ranges = ((1997, 2000), (2000, 2004), (2004, 2008), (2008, 2012), (2012, 2014))
for year_range in year_ranges:
  for year in range(*year_range):
    thread = threading.Thread(target=get_data, args=(year, "batting", "player",))
    threads.append(thread)
    thread.start()

  for thread in threads:
    thread.join()

In [52]:
threads = []
year_ranges = ((1997, 2000), (2000, 2004), (2004, 2008), (2008, 2012), (2012, 2014))
for year_range in year_ranges:
  for year in range(*year_range):
    thread = threading.Thread(target=get_data, args=(year, "pitching", "player", ))
    threads.append(thread)
    thread.start()

  for thread in threads:
    thread.join()

In [66]:
threads = []
year_ranges = ((1997, 2000), (2000, 2003), (2003, 2006), (2006, 2009), (2009, 2012), (2012, 2015), (2015, 2017))
for player_type in ["pitching", "batting"]:
  for year_range in year_ranges:
    for year in range(*year_range):
      thread = threading.Thread(target=get_data, args=(year, player_type, "teams", ))
      threads.append(thread)
      thread.start()

    for thread in threads:
      thread.join()

Exception in thread Thread-90:
Traceback (most recent call last):
  File "/Users/chrisunjae/anaconda3/envs/cs109a/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/chrisunjae/anaconda3/envs/cs109a/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-65-0630cc814e29>", line 14, in get_data
    driver.find_element_by_xpath(f"//*[@id=\"{data_type}_standard_{player_type}_sh\"]/div/ul/li[1]/div/ul/li[3]/button").click()
  File "/Users/chrisunjae/anaconda3/envs/cs109a/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py", line 80, in click
    self._execute(Command.CLICK_ELEMENT)
  File "/Users/chrisunjae/anaconda3/envs/cs109a/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py", line 633, in _execute
    return self._parent.execute(command, params)
  File "/Users/chrisunjae/anaconda3/envs/cs109a/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py